#### import functions

In [2]:
%run functions_NoiseStudy.py
%matplotlib 

# global variables
fs = 13
type_plot = ['png', 'svg']

Using matplotlib backend: MacOSX


### 0. Directory for data storage

In [3]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_measurement'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_measurement'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)

### 1. Load data

In [4]:
# [USER INPUT]
# depth profile microsensor
file_ms = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/profiles.txt'

# calibration data 
file_calib = 'Results/20210531_calibration/20210531-1455_calibration_1RoI.hdf5'

# measurement data - images of the optode
file_meas = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/'

##### additional information

In [73]:
# image cropping to the region of interest (RoI) that should be analyzed
RoI_op = [[(730, 200), (730, 1250), (1290, 1250), (1290, 200)],   # optode1
          [(1560, 200), (1560, 1250), (2100, 1250), (2100, 200)]] # optode2

# Image resolution - determine conversion factor px -> mm 
# user definition using f.ex. imageJ
px2mm, dpi = image_resolution(px=840.6646, dist_mm=30., inch=1.1811)
print('Image resolution: 1cm equals: {:.2f}px, i.e. {:.0f}dpi'.format(px2mm, dpi))

Image resolution: 1cm equals: 28.02px, i.e. 712dpi


In [74]:
# load depth profile(s) of the microsensir for validation of the depth profile
dic_micro = read_microsensor(file_ms=file_ms, encoding='latin-1')

# load measurement data and crop according to the RoI
dict_ratio = splitImage(path=file_meas, RoI_op=RoI_op)

### 2. Determine O2 concentration in each pixel and each measurement
using the Stern-Volmer equation and the respective calibration fit

In [54]:
# one example of one optode
inp = input('Which optode and which setting shall be visualized? ') # optode1, set2

Which optode and which setting shall be visualized? optode1, set2


### 3. Image blur

In [124]:
# preparation lineprofile
surface = (12.9, 12.) # optode 1, optode2
depth_lp = (-4, 4) # depth for 2D line profile 
depth_op =  (-1.5, 1.) # depth for optode excerpt
pos_lp = (7., 3.) # position of the depth profile within the optode
ls_lw = [0] # line with of the drawn depth profile

# kernel size
kernel='gauss'
ls_kernel = [(1,1), (5, 5), (11, 11), (15, 15), (21, 21), (25, 25), (31, 31), (35, 35),
             (41, 41), (45, 45), (51 ,51), (55, 55), (61, 61), (65, 65), (71, 71), 
             (75, 75), (81, 81)]  

# .................................................................
# additional information for visualization
arg = dict({'curve lw': 1.5, 'vmin':-5, 'vmax': 130, 'lw': 0.0, 'offset ms': 0.2,
           'aspect': 4, 'cmap': plt.cm.inferno, 'vmin op': 0, 'vmax op': 100,
           'figsize': (12, 5), 'fontsize': 11, 'marker': ['o', 'd', 'x', '.'],
           'colors': ['slategrey', 'darkorange', 'forestgreen', 'k']})

line profile

In [94]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_lp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v1(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dratio=dict_ratio[0], px2mm=px2mm, 
                                                ls_lw=ls_lw)), ls_kernel))

optode image

In [96]:
# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_optode = dict(map(lambda k: 
                  (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, px2mm=px2mm,
                                    kernel=kernel, dratio=dict_ratio[0], surface=surface, 
                                    depth_min=depth_op[0], depth_max=depth_op[1])),
                      ls_kernel))

### 4. Visualize image blur as line profile

In [97]:
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_lp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimages = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_lp, dO2_optode=dO2_optode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimages[k] = fig_lp

In [125]:
fig = plotLP(kshape=(51,51), dO2_lp=dO2_lp, dO2_optode=dO2_optode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])

### 5. O2 penetration depth

In [98]:
# set O2 level used for calculating the penetration depth in %air
treshold = 5.

In [100]:
# O2 penetration depth
ydepth = penetration_depth(dO2_lp=dO2_lp, ls_kernel=ls_kernel, df_ms=df_ms, 
                           treshold=treshold)

# plotting penetration depth for different kernel sizes and functions
fig_pen = plot_penetrationDepth(depth=ydepth, ls_kernel=ls_kernel, arg=arg)
plt.show()

### 6. Save results
save depth profile

In [101]:
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_' + '-'.join([i.strip() for i in inp.split(',')])

for k in dimages.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimages[k].savefig(name_DP + t, dpi=300, transparent=False)

save penetration depth

In [102]:
name_pen = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_PenetrationDepth_' + '-'.join([i.strip() for i in inp.split(',')])

for t in type_plot:
    fig_pen.savefig(name_pen + '.' + t, dpi=300, transparent=False)